In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import random
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

np.random.seed(0)

# Generate

In [3]:
def generate_sin_signal(n_time_points, n_points_in_ones):
    t = np.linspace(0, n_time_points/n_points_in_ones , n_time_points)
    w1 = np.random.normal(1, 0.01, size=(n_time_points))
    fi = np.random.uniform(-0.1, 0.1, size=n_time_points)
    signal = np.sin(t * w1 + fi)
    return signal

def generate_disc_signal(signal, win_disc):
    n_time_points = len(signal)
    new_signal = np.zeros(n_time_points)
    for i in range(0, n_time_points-1, win_disc):
        new_signal_part = signal[i:(i+win_disc)].mean()
        new_signal[i:(i+win_disc)] = new_signal_part

    return new_signal

In [4]:
n_samples = 2000
min_n_time_points = 50
max_n_time_points = 100
n_points_in_ones = 3
win_disc = 7

X_synth = np.zeros((n_samples, max_n_time_points))
y_synth = np.repeat([0, 1], 1000).reshape(2, -1).T.reshape(-1, 1)

for i_samp in range(0, n_samples, 2):

    n_time_points = int(np.random.uniform(min_n_time_points, max_n_time_points))
    
    sin_signal = generate_sin_signal(n_time_points, n_points_in_ones)
    disc_signal = generate_disc_signal(sin_signal, win_disc)

    X_synth[i_samp, :n_time_points] = sin_signal
    X_synth[i_samp + 1, :n_time_points] = disc_signal

In [5]:
synth_dataset = np.concatenate([y_synth, X_synth], axis=1)

synth_dataset = pd.DataFrame(synth_dataset)
synth_dataset = synth_dataset.sample(frac=1).reset_index(drop=True)
synth_dataset[0] = synth_dataset[0].astype(int)
synth_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0,-0.023927,0.332728,0.626686,0.828640,0.980440,0.988800,0.931743,0.765654,0.541405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-0.030262,0.262890,0.628357,0.872193,0.962486,0.990720,0.903754,0.626032,0.499180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.667132,0.667132,0.667132,0.667132,0.667132,0.667132,0.667132,-0.185633,-0.185633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,-0.035087,0.375331,0.638837,0.837876,0.991880,0.981919,0.889204,0.654002,0.515431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.097725,0.253609,0.618900,0.788993,0.961905,0.980734,0.858261,0.729795,0.353422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_size = 0.8
n_train = int(len(synth_dataset) * train_size)

train_synth = synth_dataset.iloc[:n_train]
test_synth = synth_dataset.iloc[n_train:]

In [7]:
# dir_path = 'data/TS2Vec/UCR/MySynth'
# if not os.path.exists(dir_path): 
#     os.makedirs(dir_path) 

# train_synth.to_csv(os.path.join(dir_path, 'MySynth_TRAIN.tsv'), index=None, header=None, sep='\t')
# test_synth.to_csv(os.path.join(dir_path, 'MySynth_TEST.tsv'), index=None, header=None, sep='\t')

# analysis

In [8]:
import os
import warnings

import hydra
import pandas as pd
import torch
from omegaconf import DictConfig
from torch.utils.data import DataLoader

from src.config import get_attack, get_criterion, get_disc_list, get_model
from src.data import MyDataset, load_data, transform_data
from src.estimation.estimators import AttackEstimator
from src.utils import save_experiment

/home/adversarial_attacks/miniconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Falling back on slow Cauchy and Vandermonde kernel. Install at least one of pykeops or the CUDA extension for better speed and memory efficiency.


## config

In [9]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path='../config/my_configs', version_base=None)

cfg = compose(config_name='attack_run_config.yaml')

In [10]:
device = torch.device(cfg["cuda"] if torch.cuda.is_available() else "cpu")

## load data

In [11]:
X_train, y_train, X_test, y_test = load_data(cfg['dataset'])
X_train, X_test, y_train, y_test = transform_data(X_train, X_test, y_train, y_test, slice_data=cfg['slice'])

test_loader = DataLoader(
    MyDataset(X_test, y_test), 
    batch_size=cfg['batch_size'] , 
    shuffle=False
    )


device = torch.device(cfg["cuda"] if torch.cuda.is_available() else "cpu")

attack_model_path = os.path.join(
    cfg["model_folder"],
    cfg["attack_model"]["name"],
    f"model_{cfg['model_id_attack']}_{cfg['dataset']}.pt",
)

attack_model = get_model(
    cfg["attack_model"]["name"],
    cfg["attack_model"]["params"],
    path=attack_model_path,
    device=device,
    train_mode=cfg["attack_model"]["attack_train_mode"],
)

In [12]:
criterion = get_criterion(cfg["criterion_name"], cfg["criterion_params"])

disc_check_list = (
    get_disc_list(
        model_name=cfg["disc_model_check"]["name"],
        model_params=cfg["disc_model_check"]["params"],
        list_disc_params=cfg["list_check_model_params"],
        device=device,
        path=cfg["disc_path"],
        train_mode=False,
    )
    if cfg["use_disc_check"]
    else None
)
estimator = AttackEstimator(disc_check_list, cfg["metric_effect"])

In [13]:
alpha = 0.001
eps = 0.03

In [14]:
attack_metrics = pd.DataFrame()
attack_params = dict(cfg["attack"]["attack_params"])
attack_params["model"] = attack_model
attack_params["criterion"] = criterion
attack_params["estimator"] = estimator
attack_params["alpha"] = alpha
attack_params["eps"] = eps

if "list_reg_model_params" in cfg["attack"]:
    attack_params["disc_models"] = get_disc_list(
        model_name=cfg["disc_model_reg"]["name"],
        model_params=cfg["disc_model_reg"]["params"],
        list_disc_params=cfg["attack"]["list_reg_model_params"],
        device=device,
        path=cfg["disc_path"],
        train_mode=cfg["disc_model_reg"]["attack_train_mode"],
    )

attack = get_attack(cfg["attack"]["name"], attack_params)
X_adv = attack.apply_attack(test_loader).squeeze(-1)

logging


100%|██████████| 50/50 [00:05<00:00,  9.70it/s]


In [15]:
X_adv.shape

torch.Size([400, 100])

In [16]:
lens = X_test.shape[1] - torch.sum(X_test == 0, axis=1)
lens

tensor([66, 63, 77, 65, 52, 70, 80, 56, 49, 51, 63, 76, 96, 54, 63, 64, 70, 98,
        72, 53, 70, 59, 79, 53, 61, 53, 63, 67, 80, 80, 60, 74, 51, 50, 91, 71,
        59, 59, 70, 65, 51, 66, 56, 92, 91, 50, 74, 84, 86, 76, 62, 83, 60, 90,
        70, 79, 83, 85, 68, 56, 72, 91, 83, 56, 60, 69, 98, 64, 57, 81, 97, 80,
        83, 82, 68, 89, 68, 52, 53, 63, 80, 95, 75, 77, 98, 86, 70, 95, 68, 70,
        54, 51, 52, 52, 70, 75, 53, 51, 91, 98, 89, 67, 59, 81, 59, 53, 68, 87,
        68, 84, 77, 77, 99, 79, 64, 86, 75, 63, 86, 90, 77, 74, 87, 75, 57, 79,
        97, 62, 62, 69, 95, 82, 84, 81, 66, 54, 70, 73, 65, 73, 64, 63, 78, 77,
        92, 98, 93, 68, 86, 77, 80, 97, 61, 95, 58, 96, 55, 63, 72, 82, 50, 76,
        70, 90, 55, 54, 63, 83, 68, 97, 52, 66, 91, 91, 58, 62, 66, 70, 84, 52,
        56, 80, 82, 77, 80, 97, 63, 56, 52, 79, 67, 54, 84, 63, 63, 66, 86, 70,
        61, 62, 62, 68, 85, 63, 83, 97, 87, 98, 90, 80, 69, 77, 94, 50, 76, 78,
        63, 98, 96, 63, 52, 57, 75, 93, 

In [17]:
disc_preds = estimator.calculate_hid_one_model(
    X_adv.unsqueeze(-1).to(device), 
    estimator.disc_models[0]
) \
.detach().cpu()

In [18]:
lens

tensor([66, 63, 77, 65, 52, 70, 80, 56, 49, 51, 63, 76, 96, 54, 63, 64, 70, 98,
        72, 53, 70, 59, 79, 53, 61, 53, 63, 67, 80, 80, 60, 74, 51, 50, 91, 71,
        59, 59, 70, 65, 51, 66, 56, 92, 91, 50, 74, 84, 86, 76, 62, 83, 60, 90,
        70, 79, 83, 85, 68, 56, 72, 91, 83, 56, 60, 69, 98, 64, 57, 81, 97, 80,
        83, 82, 68, 89, 68, 52, 53, 63, 80, 95, 75, 77, 98, 86, 70, 95, 68, 70,
        54, 51, 52, 52, 70, 75, 53, 51, 91, 98, 89, 67, 59, 81, 59, 53, 68, 87,
        68, 84, 77, 77, 99, 79, 64, 86, 75, 63, 86, 90, 77, 74, 87, 75, 57, 79,
        97, 62, 62, 69, 95, 82, 84, 81, 66, 54, 70, 73, 65, 73, 64, 63, 78, 77,
        92, 98, 93, 68, 86, 77, 80, 97, 61, 95, 58, 96, 55, 63, 72, 82, 50, 76,
        70, 90, 55, 54, 63, 83, 68, 97, 52, 66, 91, 91, 58, 62, 66, 70, 84, 52,
        56, 80, 82, 77, 80, 97, 63, 56, 52, 79, 67, 54, 84, 63, 63, 66, 86, 70,
        61, 62, 62, 68, 85, 63, 83, 97, 87, 98, 90, 80, 69, 77, 94, 50, 76, 78,
        63, 98, 96, 63, 52, 57, 75, 93, 

In [30]:
array_lens_preds = np.concatenate([disc_preds.numpy(), lens.unsqueeze(1)], axis=1)
df_lens_preds = pd.DataFrame(array_lens_preds, columns=['disc_preds_prob', 'lens'])
df_lens_preds['disc_preds'] = np.round(df_lens_preds['disc_preds_prob'])
df_lens_preds['quantiles'] = pd.qcut(df_lens_preds['lens'], 5, labels=False)

agg_statistic = df_lens_preds.groupby('quantiles').agg({'disc_preds_prob': 'mean', 'disc_preds': 'mean', 'lens': 'mean'})
agg_statistic = agg_statistic.rename(columns={
    'disc_preds': 'Accuracy', 
    'mean_disc_pred': 'disc_preds_prob',
    'lens': 'mean_seq_len',
    })
agg_statistic

,disc_preds_prob,Accuracy,mean_seq_len
quantiles,,,
0,1.000000,1.000000,54.259259
1,1.000000,1.000000,65.644444
2,1.000000,1.000000,75.800000
3,0.917647,0.917647,84.423529
4,0.756757,0.756757,94.837838
